In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv()
# GLOBAL CONFIGURATION
class CFG:
    # POSTGRES 
    OLLAMA_SERVER_URL=os.getenv("OLLAMA_SERVER_URL")
    DB_NAME=os.getenv("DATABASE_NAME")
    TABLE_NAME = os.getenv("VECTOR_TABLE_NAME")
    CUSTOM_TABLE_NAME="custom_theme_collection"
    PORT=os.getenv("PORT")
    CONNECTION_STRING = os.getenv("VECTOR_DATABASE_URL")

    # DIRECTORY
    EXTRACTED_FEATURE_DIR="./dataset/extracted_features"
    METADATA_DIR="./dataset/metadata"
    CACHE_FOLDER=".cache/huggingface/hub/"
    
    # EMBEDDING MODEL
    EMBEDDING_MODEL_NAME="BAAI/bge-small-en-v1.5"

### 1. Setup Embedding Configurations

In [3]:
# Setup LLM using Ollama
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3", request_timeout=120.0, base_url=CFG.OLLAMA_SERVER_URL)

In [4]:
# Setup Embedding Model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_folder=".cache/huggingface/hub/")

# Define global settings embedding model
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

/home/atuan/Desktop/chest-xray-disease-multi-classification/be-rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Load 

#### 2.1 Define PgVector store


In [5]:
import psycopg2

conn = psycopg2.connect(CFG.CONNECTION_STRING)
conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {CFG.DB_NAME}")
#     c.execute(f"CREATE DATABASE {CFG.DB_NAME}")

In [6]:
from sqlalchemy import make_url
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

url = make_url(CFG.CONNECTION_STRING)

vector_store = PGVectorStore.from_params(
    database=CFG.DB_NAME,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=CFG.TABLE_NAME,
    embed_dim=384,  # change to our 768 if using base model
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### 2.2 Load metadata & extracted features


##### 2.2.1 Create new emmbeding

In [9]:
from llama_index.core import (
    StorageContext,
    Settings,
    SimpleDirectoryReader
)

def load_data(path_to_docs='../data/documents/'):
    # Load PDF documents
    required_exts = [".pdf"]
    docs = SimpleDirectoryReader(
        input_dir=path_to_docs,   
        required_exts=required_exts,
        recursive=True,   
        filename_as_id=True 
    ).load_data()
    return docs

In [10]:
docs = load_data()
print(f"Loaded {len(docs)} documents")

Loaded 153 documents


In [11]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(docs)

In [12]:
 # https://docs.llamaindex.ai/en/stable/examples/vector_stores/postgres/#create-the-index

# If a dimension mismatch error occurs during embedding insertion, 
# ensure the database column's vector type matches the dimensions of the embeddings generated by the model.
# ex: BAAI/bge-small-en-v1.51 (dim: 384)
# ex: BAAI/bge-base-en-v1.51 (dim: 768)
# Use the following SQL command to update the column type to match the expected dimensions:
# 
# ALTER TABLE public.data_theme_colection ALTER COLUMN embedding TYPE VECTOR(768);
# 
# Note: Replace "768" with the appropriate dimension size based on the embedding model being used.
# Ensure that all existing embeddings in the column are updated accordingly to avoid inconsistency.
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, show_progress=True)
query_engine = index.as_query_engine()

Parsing nodes:   0%|          | 0/153 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 230/230 [00:03<00:00, 74.13it/s]


##### 2.2.2 Or load from existing vector database

# Querying

In [13]:
def query_chest_documents(query_engine, user_query: str):
    '''Query chest disease documents and return response in Vietnamese'''
    format_prompt = f"""
    Dựa trên tài liệu về bệnh lý ngực, hãy cung cấp thông tin về: "{user_query}"
    
    Yêu cầu:
    - Trả lời bằng tiếng Việt
    - Thông tin phải chính xác và dựa trên tài liệu
    - Nêu rõ nguồn tài liệu nếu có
    - Cung cấp thông tin đầy đủ nhưng súc tích
    """
    response = query_engine.query(format_prompt)
    print(f"Câu hỏi: {user_query}")
    print(f"Trả lời: {response}")


In [18]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize"
)

# Example queries
query_chest_documents(query_engine, "Triệu chứng của viêm phổi")
print("-"*100)
query_chest_documents(query_engine, "Phương pháp chẩn đoán ung thư phổi")
print("-"*100)
query_chest_documents(query_engine, "Cách điều trị bệnh lao phổi")

Câu hỏi: Triệu chứng của viêm phổi
Trả lời: Theo tài liệu về bệnh lý ngực, viêm phổi là một hội chứng phổ biến gây ra bởi sự nhiễm khuẩn hoặc tổn thương tại phổi. Triệu chứng của viêm phổi bao gồm:

* Đau ngực dữ dội
* Ho khạc đờm lẫn máu
* Khó thở
* Sốc (sốc cấp tính)
* Điện tâm đồ có thể thấy dấu hiệu tâm phế cấp: S sâu ở D1, Q sâu ở D3, trục phải, block nhánh phải.
* Khí máu có thể thấy tăng thông khí: PaO2 giảm và PaCO2 giảm.
* D-dimer máu tăng cao.

Nguồn tài liệu: Phac-do-VP-cong-dong-nguoi-lon_2020.pdf (trang 29) và VNRS-2022524144253.pdf (trang 47).
----------------------------------------------------------------------------------------------------
Câu hỏi: Phương pháp chẩn đoán ung thư phổi
Trả lời: Phương pháp chẩn đoán ung thư phổi bao gồm:

* X-quang phổi (Chest X-ray): Phát hiện đám mờ trên phim X-quang.
* Chụp cắt lớp vi tính lồng ngực (CT Scan of the chest): Hiển thị vị trí và kích thước của khối u.
* Sinh thiết (Biopsy): Gây mê để lấy mẫu mô phế quản, sau đó soi trực ti

In [19]:
query_chest_documents(query_engine,"Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS")

Câu hỏi: Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS
Trả lời: Tiêu chuẩn nhập khoa Điều trị tích cực của ATS bao gồm 2 tiêu chuẩn chính và 9 tiêu chuẩn phụ.

Tiêu chuẩn chính:

1. Suy hô hấp cần phải thông khí cơ học.
2. Sốc nhiễm khuẩn cần phải dùng thuốc vận mạch.

Tiêu chuẩn phụ:

1. Tần số thở > 30 lần/phút.
2. PaO2/FiO2 < 250.
3. T ổn thƣơng nhiều thùy phổi trên phim X-quang.
4. Lú lẫn, mất định hướng.
5. Ure máu (BUN > 20 mg/dL).
6. Bạch cầu máu < 4000/mm3.
7. Giảm tiểu cầu (< 100.000/mm3).
8. Hạ thân nhiệt (< 36oC).
9. Hạ huyết áp cần phải bù dịch tích cực.

Nguồn: Phac-do-VP-cong-dong-nguoi-lon_2020.pdf, trang 27.


In [20]:
!pip freeze

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


accelerate==1.2.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.11
aiosignal==1.3.2
annotated-types==0.7.0
anyio==4.8.0
asttokens==3.0.0
async-timeout==4.0.3
asyncpg==0.30.0
attrs==24.3.0
beautifulsoup4==4.12.3
certifi==2024.12.14
charset-normalizer==3.4.1
click==8.1.8
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.13
decorator==5.2.1
Deprecated==1.2.15
dirtyjson==1.0.8
distro==1.9.0
exceptiongroup==1.2.2
executing==2.2.0
fastapi==0.115.6
filelock==3.16.1
filetype==1.2.0
frozenlist==1.5.0
fsspec==2024.12.0
greenlet==3.1.1
h11==0.14.0
httpcore==1.0.7
httpx==0.27.2
httpx-sse==0.4.0
huggingface-hub==0.27.1
idna==3.10
ipykernel==6.29.5
ipython==9.1.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
Jinja2==3.1.5
jiter==0.8.2
joblib==1.4.2
jsonpatch==1.33
jsonpointer==3.0.0
jupyter_client==8.6.3
jupyter_core==5.7.2
langchain==0.3.14
langchain-community==0.3.14
langchain-core==0.3.29
langchain-experimental==0.3.4
langchain-text-splitters==0.3.5
langsmith==0.2.10
llama-cloud==0.1.7
llama